openai 1.3.2 api 调用方法

In [3]:
from openai import OpenAI
import httpx

api_key = "sk-MWCZbiYqiQUjacuGF53a6c71E3134177A585CeFe79D10aD2"
client = OpenAI(
    base_url="https://oneapi.xty.app/v1",  # 中转url
    api_key=api_key,                      # api_key
    http_client=httpx.Client(
        proxies="http://127.0.0.1:7890"  # 代理地址
    ),
)


response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "say hello to the world"}],
)

print(response.choices[0].message.content)

ChatCompletionMessage(content='Hello, world! How are you today?', role='assistant', function_call=None, tool_calls=None)


用panda读取csv文件和log文件

In [3]:
import pandas as pd

dataset = 'Spark'

with open('dataset\\' + dataset +
          '\\' + dataset + '_2k.log', 'r') as logfile:
    lines = logfile.readlines()

# print len(log_messages)
print(lines.__len__())

print(lines[0])

2000
17/06/09 20:10:40 INFO executor.CoarseGrainedExecutorBackend: Registered signal handlers for [TERM, HUP, INT]

